# Unimodal

## Requirements

Given a query, these retrieval systems selects the tracks that are most similar to the query track. The similarity is measured as cosine similarity between the feature of the query and the feature of the target track:
$$ sim(query, target) = cos(feature(query), feature(target)) $$

You will use the following features, hence implementing three different unimodal retrieval systems:

* Lyrics: `id_lyrics_bert_mmsr.tsv`
* Audio:`id_mfcc_bow_mmsr.tsv`
* Videoclips: `id_vgg19_mmsr.tsv`

In [1]:
import pandas as pd
import torch
import os, sys
from tqdm import tqdm

target_folder = 'MMSR25-26-Group-E'
current_path = os.getcwd()

while os.path.basename(current_path) != target_folder:
    if os.path.basename(current_path) == 'RetrievalAlgorithm':
        if os.path.join(current_path) not in sys.path:
            sys.path.append(os.path.join(current_path))
    parent = os.path.dirname(current_path)
    os.chdir(parent)
    current_path = parent

from RetrievalAlgorithm.src.utils.data_loading import load_all_tsv_files_from_path
from RetrievalAlgorithm.src.score_calculation_modules.cosine_similarity_module import CosineSimilarityModule
from RetrievalAlgorithm.src.unimodal_calculations import calculate_unimodal_similarity

## Module initialization

In [2]:
unimodal_module_path = 'RetrievalAlgorithm/modules/cosine_similarity_module.pt'

if os.path.exists(unimodal_module_path):
    unimodal_module = torch.jit.load(unimodal_module_path)
else:
    unimodal_module = CosineSimilarityModule()
    unimodal_module = torch.jit.script(unimodal_module)
    unimodal_module.save(unimodal_module_path)

## Data Loading

In [3]:
dataset_files_dict = load_all_tsv_files_from_path(path_to_dataset='Dataset')

Loading .tsv files: 100%|██████████| 9/9 [00:00<00:00, 611.17it/s]


## Calculating Unimodal Similarity Scores

In [4]:
from RetrievalAlgorithm.src.normalization import *


norm_names_types_list = [
    ('raw', NormalizationModule),
    ('min_max', MinMaxNormalizationModule),
    ('max_abs', MaxAbsNormalizationModule),
    ('standard', StandardNormalizationModule),
    ('robust', RobustNormalizationModule),
]
sim_scores_df_list = []

### Lyrics

In [5]:
for norm_name, norm_module_type in norm_names_types_list:
    print('='*100)
    print('Normalization name:', norm_name)
    lyrics_similarity_scores_df = calculate_unimodal_similarity(
        dataset_df=dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
        calculation_module=unimodal_module,
        normalization_module_type=norm_module_type,
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'lyrics', lyrics_similarity_scores_df))

Normalization name: raw


Batches: 100%|██████████| 8404/8404 [01:58<00:00, 70.81it/s] 


Normalization name: min_max


Batches: 100%|██████████| 8404/8404 [02:04<00:00, 67.27it/s] 


Normalization name: max_abs


Batches: 100%|██████████| 8404/8404 [02:00<00:00, 69.63it/s] 


Normalization name: standard


Batches: 100%|██████████| 8404/8404 [02:06<00:00, 66.65it/s] 


Normalization name: robust


Batches: 100%|██████████| 8404/8404 [02:01<00:00, 69.39it/s] 


### Audio

In [6]:
for norm_name, norm_module_type in norm_names_types_list:
    print('='*100)
    print('Normalization name:', norm_name)

    audio_similarity_scores_df = calculate_unimodal_similarity(
        dataset_df=dataset_files_dict['id_mfcc_bow_mmsr.tsv'],
        calculation_module=unimodal_module,
        normalization_module_type=norm_module_type,
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'audio', audio_similarity_scores_df))

Normalization name: raw


Batches: 100%|██████████| 8404/8404 [01:44<00:00, 80.61it/s] 


Normalization name: min_max


Batches: 100%|██████████| 8404/8404 [01:48<00:00, 77.22it/s] 


Normalization name: max_abs


Batches: 100%|██████████| 8404/8404 [01:55<00:00, 73.07it/s] 


Normalization name: standard


Batches: 100%|██████████| 8404/8404 [02:02<00:00, 68.69it/s] 


Normalization name: robust


Batches: 100%|██████████| 8404/8404 [01:54<00:00, 73.15it/s] 


### Videoclips

In [7]:
for norm_name, norm_module_type in norm_names_types_list:
    print('='*100)
    print('Normalization name:', norm_name)

    videoclips_similarity_scores_df = calculate_unimodal_similarity(
        dataset_df=dataset_files_dict['id_vgg19_mmsr.tsv'],
        calculation_module=unimodal_module,
        normalization_module_type=norm_module_type,
        batch_size=1024,
        include_reverse_pairs=False,
        include_self_pairs=True,
    )

    sim_scores_df_list.append((norm_name, 'video', videoclips_similarity_scores_df))

Normalization name: raw


Batches: 100%|██████████| 8404/8404 [09:13<00:00, 15.18it/s]


Normalization name: min_max


Batches: 100%|██████████| 8404/8404 [11:53<00:00, 11.78it/s]


Normalization name: max_abs


Batches: 100%|██████████| 8404/8404 [11:06<00:00, 12.60it/s]


Normalization name: standard


Batches: 100%|██████████| 8404/8404 [12:39<00:00, 11.06it/s]


Normalization name: robust


Batches: 100%|██████████| 8404/8404 [12:13<00:00, 11.46it/s]


## Merge and Save results

In [8]:
target_dir = 'RetrievalAlgorithm/results/unimodal'
os.makedirs(target_dir, exist_ok=True)

for norm_name, feature_name, sim_scores_df in tqdm(sim_scores_df_list, desc='Saving similarity scores'):
    output_path = os.path.join(target_dir, norm_name)
    os.makedirs(output_path, exist_ok=True)
    output_path = os.path.join(output_path, f'unimodal_{norm_name}_{feature_name}_similarity_scores.parquet')
    sim_scores_df.to_parquet(output_path, index=False)

Saving similarity scores: 100%|██████████| 15/15 [01:08<00:00,  4.57s/it]
